In [73]:
import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import pickle

In [5]:
# load sample data
path = "/Users/paraspokharel/Programming/costomFPL/costomFPL/data/football_data_uk/Bundesliga_16_17.csv"
data = pd.read_csv(path)
data

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,h_course,d_course,a_course,h_pts_avg3,...,a_pozycja,a_ATT,a_MID,a_DEF,a_OVR,position_dst,ATT_dst,MID_dst,DEF_dst,OVR_dst
0,2016-08-27,Augsburg,Wolfsburg,0,2,A,2.80,3.25,2.60,1.000000,...,NaN,77.0,78.0,76.0,77.0,NaN,4.0,4.0,2.0,3.0
1,2016-08-27,Dortmund,Mainz,2,1,H,1.36,5.00,8.50,2.333333,...,NaN,74.0,74.0,75.0,74.0,NaN,9.0,7.0,6.0,7.0
2,2016-08-27,Ein Frankfurt,Schalke 04,1,0,H,3.40,3.30,2.20,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-08-27,FC Koln,Darmstadt,2,0,H,1.80,3.50,4.75,2.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-08-27,Hamburg,Ingolstadt,1,1,D,2.20,3.20,3.50,1.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,2017-05-20,Hamburg,Wolfsburg,2,1,H,2.60,3.40,2.70,1.666667,...,15.0,77.0,78.0,76.0,77.0,1.0,NaN,NaN,NaN,NaN
301,2017-05-20,Hertha,Leverkusen,2,6,A,2.05,3.40,3.80,1.000000,...,12.0,83.0,79.0,78.0,79.0,6.0,9.0,4.0,4.0,5.0
302,2017-05-20,Hoffenheim,Augsburg,0,0,D,1.36,5.00,8.50,1.333333,...,15.0,73.0,74.0,74.0,74.0,11.0,5.0,2.0,2.0,3.0
303,2017-05-20,Ingolstadt,Schalke 04,1,1,D,2.63,3.50,2.60,1.000000,...,10.0,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN


In [30]:
# prepare training data

label_map = {"H":0, "D":1, "A":2}
x_df = data.drop(columns = ["HomeTeam", "AwayTeam", "Date", "FTR"])
y_df = data["FTR"].map(label_map)
x_train, x_test = train_test_split(x_df, test_size = 0.3, random_state = 42)
y_train, y_test = train_test_split(y_df, test_size = 0.3, random_state = 42)

,FTHG,FTAG,h_course,d_course,a_course,h_pts_avg3,h_pts_avg5,h_pts_avg7,h_pts_std3,h_pts_std5,...,a_pozycja,a_ATT,a_MID,a_DEF,a_OVR,position_dst,ATT_dst,MID_dst,DEF_dst,OVR_dst
72,1,3,15.00,6.5,1.20,1.000000,1.2,1.285714,0.000000,0.979796,...,1.0,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN
15,4,4,2.00,3.5,3.75,1.333333,1.0,1.142857,1.247219,1.095445,...,16.0,78.0,76.0,76.0,77.0,1.0,4.0,2.0,1.0,3.0
10,3,1,3.60,3.6,2.00,1.333333,1.4,1.428571,1.247219,1.356466,...,11.0,78.0,77.0,79.0,78.0,1.0,5.0,3.0,3.0,3.0
164,1,0,2.05,3.4,3.75,2.333333,1.8,1.285714,0.942809,0.979796,...,14.0,77.0,78.0,76.0,77.0,7.0,NaN,NaN,NaN,NaN
224,1,3,2.90,3.1,2.60,1.000000,1.2,1.285714,1.414214,1.469694,...,4.0,78.0,76.0,76.0,77.0,1.0,4.0,1.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,1,2,1.91,3.5,4.20,1.333333,0.8,1.000000,1.247219,1.166190,...,16.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
71,0,0,1.85,3.5,4.50,1.666667,1.0,1.571429,0.942809,1.095445,...,7.0,76.0,77.0,75.0,76.0,3.0,2.0,0.0,4.0,2.0
106,3,1,1.33,5.0,10.00,0.666667,1.2,1.285714,0.471405,0.979796,...,15.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN
270,3,0,3.30,3.3,2.25,1.000000,1.8,1.285714,1.414214,1.469694,...,6.0,73.0,74.0,76.0,75.0,12.0,NaN,NaN,NaN,NaN


In [60]:
# train xgboost

model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.1, max_depth=6, n_estimators=100)

# Fit the model with early stopping and evaluation metric
model.fit(x_train, y_train,
          eval_set=[(x_train, y_train), (x_test, y_test)],    
          verbose=False) 
# TODO: optimize

In [71]:
# evaluate
eval_result = model.evals_result()
pred_y = model.predict_proba(x_test)
auc =  roc_auc_score(y_test, pred_y, multi_class = "ovo")

np.float64(0.9990119529593214)

In [78]:
# download model (pickle converts to bytestream)
pickle.dump(model, open(r"../models/xgb_model.pkl", "wb"))